# 3rd Place Solution


## Cryptocurrency Closing Price Prediction Challenge


Can you predict the closing price for a cryptocurrency? 

The objective of this challenge is to create a machine learning model that will predict closing price for a crypto currency coing. 

In [ ]:
!pip install linear-tree

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from google.colab import files
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from lineartree import LinearTreeRegressor
from sklearn.linear_model import Lasso,Ridge,BayesianRidge
from sklearn.ensemble import StackingRegressor,GradientBoostingRegressor,RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold

In [ ]:
# Load files
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
ss = pd.read_csv('SampleSubmission.csv')

In [ ]:
# check data shapes
train.shape, test.shape, ss.shape

((12632, 49), (6222, 48), (6222, 2))

In [ ]:
# Preview train
train.head()

,id,asset_id,open,high,low,volume,market_cap,url_shares,unique_url_shares,reddit_posts,reddit_posts_score,reddit_comments,reddit_comments_score,tweets,tweet_spam,tweet_followers,tweet_quotes,tweet_retweets,tweet_replies,tweet_favorites,tweet_sentiment1,tweet_sentiment2,tweet_sentiment3,tweet_sentiment4,tweet_sentiment5,tweet_sentiment_impact1,tweet_sentiment_impact2,tweet_sentiment_impact3,tweet_sentiment_impact4,tweet_sentiment_impact5,social_score,average_sentiment,news,price_score,social_impact_score,correlation_rank,galaxy_score,volatility,market_cap_rank,percent_change_24h_rank,volume_24h_rank,social_volume_24h_rank,social_score_24h_rank,medium,youtube,social_volume,percent_change_24h,market_cap_global,close
0,ID_322qz6,1,9422.849081,9428.490628,9422.849081,7.131986e+08,1.737635e+11,1689.0,817.0,55.0,105.0,61.0,271.0,3420.0,1671.0,11675867.0,39.0,1343.0,448.0,2237.0,124.0,330.0,331.0,2515.0,120.0,506133.0,1326610.0,1159677.0,8406185.0,281329.0,11681999.0,3.6,69.0,2.7,3.6,3.3,66.0,0.007118,1.0,606.0,2.0,1.0,1.0,2.0,5.0,4422,1.434516,2.818066e+11,9428.279323
1,ID_3239o9,1,7985.359278,7992.059917,7967.567267,4.004755e+08,1.426942e+11,920.0,544.0,20.0,531.0,103.0,533.0,1491.0,242.0,5917814.0,195.0,1070.0,671.0,3888.0,1.0,52.0,315.0,1100.0,23.0,1320.0,381117.0,1706376.0,3754815.0,80010.0,5924770.0,3.7,1.0,2.0,2.0,1.0,43.5,0.009419,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2159,-2.459507,2.126897e+11,7967.567267
2,ID_323J9k,1,49202.033778,49394.593518,49068.057046,3.017729e+09,9.166977e+11,1446.0,975.0,72.0,1152.0,187.0,905.0,9346.0,4013.0,47778746.0,104.0,2014.0,1099.0,11476.0,331.0,923.0,864.0,6786.0,442.0,9848462.0,5178557.0,2145663.0,25510267.0,5110490.0,47796942.0,3.7,22.0,3.1,3.0,3.3,65.5,0.013530,1.0,692.0,3.0,1.0,1.0,NaN,NaN,10602,4.942448,1.530712e+12,49120.738484
3,ID_323y5P,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,424.0,268.0,443.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,285,NaN,NaN,NaN
4,ID_324kJH,1,10535.737119,10535.737119,10384.798216,1.150053e+09,1.921183e+11,1012.0,638.0,24.0,42.0,50.0,173.0,3262.0,1652.0,14422172.0,21.0,511.0,190.0,2284.0,86.0,280.0,443.0,2284.0,169.0,311017.0,1977833.0,731277.0,10964321.0,440730.0,14426405.0,3.7,22.0,4.7,3.8,4.4,83.0,0.010332,1.0,749.0,2.0,1.0,1.0,NaN,2.0,3996,2.609576,3.386925e+11,10384.798216


In [ ]:
train = train.fillna(0)
test = test.fillna(0)

In [ ]:
train=train[train.close!=0]
train=train[train.close<=train.high]

In [ ]:
train=train[train['open']<=61158.881991]

In [ ]:
train.reset_index(drop=True,inplace=True)

In [ ]:
intersec=test[test['open'].isin(train.open)].open

In [ ]:
main_cols=["open", "high", "low", "market_cap", "tweets", "social_volume", "tweet_sentiment2", 
           "social_impact_score", "tweet_spam", 'volatility', "percent_change_24h_rank", 
           "volume_24h_rank", "tweet_retweets"]

kmeans = KMeans(n_clusters=4,random_state=1)
train['Cluster'] = kmeans.fit_predict(train[['open', "high", "low"]])
test['Cluster']=kmeans.predict(test[["open", "high", "low"]])

In [ ]:
# Select main columns to be used in training
main_cols=["open","high","low","market_cap","tweets","social_volume",
           "tweet_sentiment2","social_impact_score","tweet_spam",'volatility',
           "percent_change_24h_rank","volume_24h_rank","tweet_retweets","Cluster"]

In [ ]:
params={'criterion': 'mse',
 'max_bins': 32,
 'max_depth': 2,
 'min_samples_leaf': 4,
 'min_samples_split': 10
 }

In [ ]:
n=5
skf = KFold(n_splits=n, shuffle=True, random_state=5168)

In [ ]:
oof=[]
val=[]
for fold, (trn_idx, val_idx) in enumerate(skf.split(train, train['close'])):
    X_train, y_train = train.iloc[trn_idx][main_cols], train.iloc[trn_idx]['close']
    X_valid, y_valid = train.iloc[val_idx][main_cols], train.iloc[val_idx]['close']
    model = LinearTreeRegressor(LinearRegression(),**params)
    model.fit(X_train, y_train)
    val.append(mean_squared_error(y_valid,model.predict(X_valid),squared=False))
    oof.append(model.predict(test[main_cols]))

In [ ]:
np.mean(val)

63.771827581598714

In [ ]:
out_sum = oof[0]
for i in range(1,len(oof)):
    out_sum = oof[i] + out_sum

In [ ]:
pred=(out_sum/len(oof))

In [ ]:
test['close']=pred

In [ ]:
test['close']=np.clip(pred,test['low'],test["high"])

In [ ]:
intersec.reset_index(drop=True,inplace=True)

In [ ]:
for i in intersec:
  test.loc[test.open==i,'close']=train[train.open==i].close.values[0]

In [ ]:
sub_file = ss.copy()
sub_file.close = test.close
sub_file.to_csv('Sub_'+str(np.mean(val))+'.csv', index = False)

In [ ]:
sub_file.head()

,id,close
0,ID_323Sn2,0.000000
1,ID_325SNW,11351.349066
2,ID_325uzE,6307.326575
3,ID_328qCx,0.000000
4,ID_3293uJ,0.000000


In [ ]:
files.download('Sub_'+str(np.mean(val))+'.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>